In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt

## Import Apple and Google Mobility Data

In [2]:
# Create paths to CSVs
google_path = '../Resources/Global_Mobility_Report.csv'
apple_path = '../Resources/applemobilitytrends-2020-04-13.csv'

In [3]:
# Create DFs
google_df = pd.DataFrame(pd.read_csv(google_path))
apple_df = pd.DataFrame(pd.read_csv(apple_path))

C:\Users\Rob-Desktop\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Clean up Apple Data

In [4]:
# Find data for country. Remove city data.
apple_df = apple_df.loc[apple_df["geo_type"]=="country/region"].copy()

# Group by region to get the average mobility type together, and copy
apple_df = apple_df.groupby("region").mean().copy()
apple_df.head()

,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,2020-01-21,2020-01-22,...,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13
region,,,,,,,,,,,,,,,,,,,,,
Albania,100,97.990000,100.180000,97.830000,102.200000,106.400000,93.480000,95.020000,95.925000,94.450000,...,22.720000,23.230,28.765000,25.915000,28.120000,27.830000,26.805000,26.550000,27.240000,34.080000
Argentina,100,96.090000,101.910000,111.940000,117.585000,119.075000,89.990000,98.250000,100.770000,101.285000,...,13.050000,10.325,14.545000,16.020000,17.730000,17.225000,15.190000,17.575000,11.670000,18.725000
Australia,100,102.023333,102.223333,104.243333,107.050000,92.070000,99.263333,105.066667,108.146667,106.103333,...,27.023333,38.510,39.026667,39.596667,42.576667,27.960000,29.590000,24.073333,28.426667,40.183333
Austria,100,101.345000,104.915000,112.225000,120.295000,124.135000,97.550000,100.650000,102.985000,104.805000,...,32.320000,39.355,40.470000,43.215000,44.280000,45.250000,42.785000,37.790000,37.270000,38.125000
Belgium,100,100.440000,107.646667,110.800000,123.343333,135.013333,104.976667,104.790000,110.530000,114.663333,...,41.396667,45.140,40.213333,45.513333,47.256667,47.573333,50.393333,53.890000,45.793333,37.536667


In [5]:
# Reset index
apple_df = apple_df.reset_index()

In [6]:
# Melt Apple data df, to transform data structure.
# Goal is to have dates listed by column, to merge later with Google data.
apple_df = apple_df.melt(id_vars=["region"],
            var_name="date",
            value_name="value")

In [7]:
# Sort values by Country and by Date (chronological)
apple_df = apple_df.sort_values(by=["region", "date"], ascending=True)

# Rename columns
apple_df = apple_df.rename(columns={
                           "region": "country",
                           "date": "dates",
                           "value": "mobility_percent_from_base"
})

apple_df.head()

,country,dates,mobility_percent_from_base
0,Albania,2020-01-13,100.00
63,Albania,2020-01-14,97.99
126,Albania,2020-01-15,100.18
189,Albania,2020-01-16,97.83
252,Albania,2020-01-17,102.20


In [8]:
# Reset index
apple_df = apple_df.reset_index(drop=True)

apple_df.head()

,country,dates,mobility_percent_from_base
0,Albania,2020-01-13,100.00
1,Albania,2020-01-14,97.99
2,Albania,2020-01-15,100.18
3,Albania,2020-01-16,97.83
4,Albania,2020-01-17,102.20


In [9]:
# Rename/drop countries for better merging compatibility

# Set index to country
apple_df = apple_df.set_index("country")

# Rename Korea/UK
apple_df = apple_df.rename(index={"Republic of Korea":"South Korea", "UK":"United Kingdom"})

# Reset index and drop Czech Replublic
apple_df = apple_df.reset_index()
apple_df = apple_df.loc[apple_df["country"]!="Czech Republic"]

apple_df.head()

,country,dates,mobility_percent_from_base
0,Albania,2020-01-13,100.00
1,Albania,2020-01-14,97.99
2,Albania,2020-01-15,100.18
3,Albania,2020-01-16,97.83
4,Albania,2020-01-17,102.20


### Clean up Google Data

In [10]:
# A value in the "sub_region_1" indicates mobility data for a given region/city data within a country.
# NaN in the "sub_region_1" category indicates that row shows data for the whole country.
# We want to compare country-level only, so we will drop the rows containing any city/region data.

# Drop rows where 'sub_region_1' does NOT equal 'NaN'
google_nan = google_df.dropna(subset=['sub_region_1'])
google_df = google_df.drop(google_nan.index).copy()
google_df

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
5,AE,United Arab Emirates,NaN,NaN,2020-02-20,-2.0,1.0,6.0,1.0,1.0,1.0
6,AE,United Arab Emirates,NaN,NaN,2020-02-21,-3.0,2.0,6.0,0.0,-1.0,1.0
7,AE,United Arab Emirates,NaN,NaN,2020-02-22,-2.0,2.0,4.0,-2.0,3.0,1.0
8,AE,United Arab Emirates,NaN,NaN,2020-02-23,-1.0,3.0,3.0,-1.0,4.0,1.0
9,AE,United Arab Emirates,NaN,NaN,2020-02-24,-3.0,0.0,5.0,-1.0,3.0,1.0


In [11]:
# Google lists mobility data across several categories (retail, groceries, parks, etc.).
# Find average of all mobility data, to compare better to Apple's data.

google_df['avg_all_change'] = ((google_df['retail_and_recreation_percent_change_from_baseline'] + 
google_df['grocery_and_pharmacy_percent_change_from_baseline'] + 
google_df['parks_percent_change_from_baseline'] +
google_df['transit_stations_percent_change_from_baseline'] +
google_df['workplaces_percent_change_from_baseline'] +
google_df['residential_percent_change_from_baseline'])/6)
google_df.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,avg_all_change
0,AE,United Arab Emirates,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0,2.000000
1,AE,United Arab Emirates,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0,2.166667
2,AE,United Arab Emirates,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0,1.500000
3,AE,United Arab Emirates,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0,1.166667
4,AE,United Arab Emirates,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0,0.666667


In [12]:
# Drop remaining unnecessary columns
google_avg_df = google_df.drop(columns=['retail_and_recreation_percent_change_from_baseline',
                                        'grocery_and_pharmacy_percent_change_from_baseline',
                                        'parks_percent_change_from_baseline',
                                        'transit_stations_percent_change_from_baseline',
                                        'workplaces_percent_change_from_baseline',
                                        'residential_percent_change_from_baseline',
                                        'country_region_code',
                                        'sub_region_1',
                                        'sub_region_2'])
google_avg_df.head()

,country_region,date,avg_all_change
0,United Arab Emirates,2020-02-15,2.000000
1,United Arab Emirates,2020-02-16,2.166667
2,United Arab Emirates,2020-02-17,1.500000
3,United Arab Emirates,2020-02-18,1.166667
4,United Arab Emirates,2020-02-19,0.666667


In [13]:
# Sort and reset index
google_avg_df = (google_avg_df.sort_values(by=['country_region','date'])).reset_index()
google_avg_df = google_avg_df.drop(columns=['index'])

In [14]:
# Convert percentage of baseline to be compatible with Apple percent of baseline
google_avg_df["avg_all_change"]+=100
google_avg_df.head()

,country_region,date,avg_all_change
0,Afghanistan,2020-02-15,92.666667
1,Afghanistan,2020-02-16,104.833333
2,Afghanistan,2020-02-17,105.333333
3,Afghanistan,2020-02-18,104.166667
4,Afghanistan,2020-02-19,101.500000


In [15]:
# Rename columns
google_avg_df = google_avg_df.rename(columns={
                           "country_region": "country",
                           "date": "dates",
                           "avg_all_change": "mobility_percent_from_base"    
})
google_avg_df.head()

,country,dates,mobility_percent_from_base
0,Afghanistan,2020-02-15,92.666667
1,Afghanistan,2020-02-16,104.833333
2,Afghanistan,2020-02-17,105.333333
3,Afghanistan,2020-02-18,104.166667
4,Afghanistan,2020-02-19,101.500000


In [16]:
# Drop NA cells.
google_avg_df = google_avg_df.dropna(how="any")
google_avg_df.head()

,country,dates,mobility_percent_from_base
0,Afghanistan,2020-02-15,92.666667
1,Afghanistan,2020-02-16,104.833333
2,Afghanistan,2020-02-17,105.333333
3,Afghanistan,2020-02-18,104.166667
4,Afghanistan,2020-02-19,101.500000


In [17]:
# Drop countries for better merging compatibility
google_avg_df = google_avg_df.loc[google_avg_df["country"]!="Cape Verde"]
google_avg_df = google_avg_df.loc[google_avg_df["country"]!="Czechia"]
google_avg_df = google_avg_df.loc[google_avg_df["country"]!="Myanmar (Burma)"]
google_avg_df = google_avg_df.loc[google_avg_df["country"]!="The Bahamas"]
google_avg_df.head()

,country,dates,mobility_percent_from_base
0,Afghanistan,2020-02-15,92.666667
1,Afghanistan,2020-02-16,104.833333
2,Afghanistan,2020-02-17,105.333333
3,Afghanistan,2020-02-18,104.166667
4,Afghanistan,2020-02-19,101.500000


### Clean COVID-19 data

In [18]:
# Import CSV
covid_path = '../Resources/full_data.csv'
covid_df = pd.DataFrame(pd.read_csv(covid_path))
covid_df.head()

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2020-02-25,Afghanistan,NaN,NaN,1,NaN
1,2020-02-26,Afghanistan,0.0,NaN,1,NaN
2,2020-02-27,Afghanistan,0.0,NaN,1,NaN
3,2020-02-28,Afghanistan,0.0,NaN,1,NaN
4,2020-02-29,Afghanistan,0.0,NaN,1,NaN


In [19]:
# Reorder columns
covid_df = covid_df[["location", "date", "new_cases", "total_cases", "new_deaths", "total_deaths"]]
covid_df.head()

,location,date,new_cases,total_cases,new_deaths,total_deaths
0,Afghanistan,2020-02-25,NaN,1,NaN,NaN
1,Afghanistan,2020-02-26,0.0,1,NaN,NaN
2,Afghanistan,2020-02-27,0.0,1,NaN,NaN
3,Afghanistan,2020-02-28,0.0,1,NaN,NaN
4,Afghanistan,2020-02-29,0.0,1,NaN,NaN


In [20]:
# Fill NA values as 0
covid_df = covid_df.fillna(int(0))

# Cast 0 values as integer
covid_df = covid_df.astype({"new_cases": "int64"})
covid_df = covid_df.astype({"total_cases": "int64"})
covid_df = covid_df.astype({"new_deaths": "int64"})
covid_df = covid_df.astype({"total_deaths": "int64"})

# Rename column
covid_df = covid_df.rename(columns={
                           "location": "country",
                           "date": "dates"
})

covid_df.head()

,country,dates,new_cases,total_cases,new_deaths,total_deaths
0,Afghanistan,2020-02-25,0,1,0,0
1,Afghanistan,2020-02-26,0,1,0,0
2,Afghanistan,2020-02-27,0,1,0,0
3,Afghanistan,2020-02-28,0,1,0,0
4,Afghanistan,2020-02-29,0,1,0,0


In [21]:
# Drop countries for better merging compatibility
covid_df = covid_df.loc[covid_df["country"]!="Cote d'Ivoire"]
covid_df = covid_df.loc[covid_df["country"]!="Czech Republic"]
covid_df = covid_df.loc[covid_df["country"]!="Democratic Republic of Congo"]
covid_df = covid_df.loc[covid_df["country"]!="Guernsey"]
covid_df = covid_df.loc[covid_df["country"]!="International"]
covid_df = covid_df.loc[covid_df["country"]!="Macedonia"]
covid_df = covid_df.loc[covid_df["country"]!="Saint Barthlemy"]
covid_df = covid_df.loc[covid_df["country"]!="Saint Martin (French part)"]
covid_df = covid_df.loc[covid_df["country"]!="Worldwide"]
covid_df = covid_df.loc[covid_df["country"]!="Vatican"]
covid_df = covid_df.loc[covid_df["country"]!="Palestine"]

covid_df.head()

,country,dates,new_cases,total_cases,new_deaths,total_deaths
0,Afghanistan,2020-02-25,0,1,0,0
1,Afghanistan,2020-02-26,0,1,0,0
2,Afghanistan,2020-02-27,0,1,0,0
3,Afghanistan,2020-02-28,0,1,0,0
4,Afghanistan,2020-02-29,0,1,0,0


### Clean population data

In [22]:
# Import population data
population_path = '../Resources/population_by_country_2020.csv'
population_df = pd.DataFrame(pd.read_csv(population_path))
population_df.head()

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1438207241,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1377233523,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,330610570,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,272931713,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,219992900,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %


In [23]:
# Include only desired columns (country name and population)
population_df = population_df[['Country (or dependency)','Population (2020)']]

# Rename column
population_df = population_df.rename(columns={
                                   "Country (or dependency)": "country",
                                   "Population (2020)": "population"
})

population_df.head()

,country,population
0,China,1438207241
1,India,1377233523
2,United States,330610570
3,Indonesia,272931713
4,Pakistan,219992900


### Join Google and Apple Datasets

In [24]:
# Round data in mobility percent column
google_avg_df["mobility_percent_from_base"] = round(google_avg_df["mobility_percent_from_base"],2)
apple_df["mobility_percent_from_base"] = round(apple_df["mobility_percent_from_base"],2)

In [25]:
# Merge Google and Apple Data.
# Perform inner join on country and date columns
mobility_df = pd.merge(apple_df, google_avg_df, how='inner', on=["country", "dates"])

# Rename
mobility_df = mobility_df.rename(columns={
    "mobility_percent_from_base_x": "apple_mobility_percent",
    "mobility_percent_from_base_y": "google_mobility_percent"
})
mobility_df.head()

,country,dates,apple_mobility_percent,google_mobility_percent
0,Argentina,2020-02-15,115.54,99.67
1,Argentina,2020-02-16,78.66,95.00
2,Argentina,2020-02-17,92.85,94.17
3,Argentina,2020-02-18,104.82,103.50
4,Argentina,2020-02-19,102.99,104.67


# Python SQL toolkit and Object Relational Mapper

In [26]:
# Import dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, select, join

from config import password

In [27]:
# Create engine
# Note: User will need to supply their own PostgreSQL password under variable below

user = "postgres"
host = "localhost"
# password = "INSERT_YOUR_PASSWORD_HERE"
port = "5432"
db = "Covid_mobility"
uri = f"postgresql://{user}:{password}@{host}:{port}/{db}"
uri
engine = create_engine(uri)

In [28]:
# Export content to SQL Tables (previously created in SQL)
population_df.to_sql(name="countries", con=engine, if_exists="replace", index=False)
mobility_df.to_sql(name="apple_google_mobility", con=engine, if_exists="replace", index=False)
covid_df.to_sql(name="infections", con=engine, if_exists="replace", index=False)